In [ ]:
import os, subprocess, sys
gpu = subprocess.run('nvidia-smi', shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL).returncode == 0
req = 'requirements-colab-gpu.txt' if gpu else 'requirements-colab-cpu.txt'
subprocess.run(['apt-get','-y','update'], check=True)
subprocess.run(['apt-get','-y','install','ffmpeg'], check=True)
subprocess.run([sys.executable,'-m','pip','install','-r',req], check=True)
if gpu:
    subprocess.run([sys.executable,'-m','pip','install','torch','torchaudio','torchvision','--index-url','https://download.pytorch.org/whl/cu121'], check=True)
else:
    subprocess.run([sys.executable,'-m','pip','install','torch','torchaudio','torchvision','--index-url','https://download.pytorch.org/whl/cpu'], check=True)


In [ ]:
import math, wave, array, subprocess, os
from pathlib import Path

def _write_tone(path, freq, duration=5, sr=44100):
    t = [math.sin(2*math.pi*freq*i/sr) for i in range(int(duration*sr))]
    ints = array.array('h', [int(max(-1.0,min(1.0,x))*32767) for x in t])
    path.parent.mkdir(parents=True, exist_ok=True)
    with wave.open(str(path), 'wb') as wf:
        wf.setnchannels(1); wf.setsampwidth(2); wf.setframerate(sr); wf.writeframes(ints.tobytes())

def make_demo(directory):
    freqs={'vocals':440,'drums':220,'bass':110,'other':330}
    for name,f in freqs.items():
        _write_tone(directory / f'{name}.wav', f)

inp = Path('demo_stems')
out = Path('demo_output')
make_demo(inp)
subprocess.run([sys.executable, 'scripts/mix_cli.py', str(inp), str(out)], check=True)
print('Mixed files:', os.listdir(out))
